In [1]:
!pip install anthropic

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [4]:
import anthropic
import base64
import os

api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise ValueError("ANTHROPIC_API_KEY environment variable is not set")
client = anthropic.Client(api_key=api_key)
# Support sending images with the message
# Example with a local image file
with open("docs/3.png", "rb") as f:
    image_data = base64.b64encode(f.read()).decode('utf-8')
    
message = """
You are a professional screenshot analyzer for lawyers. 
Could you, please, say, what site is visited on this screenshot and summarize a user activity.
Write only summary of the activity in a form of 
"Reserching the doc "Name of the doc, article" or "writing the note on "name of the case".
"""

# Send a message to Claude with both text and image
response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0.7,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": message},
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": image_data
                    }
                }
            ]
        }
    ]
)


# Print Claude's response
print(response.content[0].text)

Researching the doc "Code de commerce: Chapitre III - Des obligations générales des commerçants (Articles L123-1 à L123-57)" on Legifrance.gouv.fr, focusing on sections about commercial registry management and registration effects.


In [9]:
from bs4 import BeautifulSoup

system_prompt = """
You are an AI assistant tasked with summarizing what a lawyer is currently doing based on the HTML content of the page they are viewing. Your goal is to provide a brief, concise summary in the format: "Action: [brief description]".
Analyze the HTML content and follow these steps:

1. Look for key indicators of the lawyer's activity, such as:
   - Page title or header
   - Document names or case names
   - Article titles
   - Names of individuals or organizations
   - Keywords related to legal activities (e.g., "research", "brief", "motion", "deposition")

2. Based on these indicators, determine the most likely activity the lawyer is engaged in. Common activities might include:
   - Researching a case
   - Writing a legal document
   - Reviewing case materials
   - Preparing for a call or meeting

3. Summarize the activity in a brief phrase, following this format:
   - For case research: "Researching case '[case name]'"
   - For article reading: "Reading article '[article title]'"
   - For document writing: "Writing [document type] for '[case or client name]'"
   - For call preparation: "Preparing for call with '[individual or organization name]'"

4. If the specific activity is unclear, use a more general description, such as "Reviewing legal materials" or "Accessing case information".

Provide your summary in the following format:

<summary>
Action: [Your brief description of the lawyer's activity]
</summary>

Remember to keep the summary concise and focused on the main activity indicated by the HTML content. Do not include any additional explanations or details outside of the <summary> tags.
"""
# Read HTML content from file
with open("docs/1.html", "r", encoding="utf-8") as f:
    HTML_CONTENT = f.read()

user_prompt = """Here is the HTML content of the page the lawyer is currently viewing:
<html_content>
{{HTML_CONTENT}}
</html_content>
"""
# Parse HTML with BeautifulSoup
soup = BeautifulSoup(HTML_CONTENT, 'html.parser')

# Remove script and style elements
for script in soup(["script", "style"]):
    script.decompose()

# Remove comments
#for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
#    comment.extract()

# Get text content
HTML_CONTENT = soup.get_text(strip=True)
# Send message to Claude with prompts and image
#user_prompt = user_prompt.replace("{{HTML_CONTENT}}", HTML_CONTENT)
user_prompt = user_prompt.replace("{{HTML_CONTENT}}", HTML_CONTENT)
response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0.7,
    system=system_prompt,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": user_prompt},
            ]
        }
    ]
)
     

# Print Claude's response 
print(response.content[0].text)

<summary>
Action: Reading section III of the French Commercial Code regarding general obligations of merchants (Articles L123-1 to L123-57)
</summary>
